In [9]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [37]:
def get_data(n, amplitude=1, dataset='dataset_1'):
    '''
    returns the maps with and without cosmic string signal with appropriate labels (index matching).
    
    Parameters:
    --------------
    n : number of maps to be fetched for each type of map (thus it returns 2*l maps)
    
    amplitude : multiplicative factor used to augment the cosmic string signal
    
    dataset : the dataset from which the data is pulled
    
    '''
    #load the data
    path_ns = "../data/" + dataset + "/ns-maps/"
    path_ws = "../data/" + dataset + "/ws-maps/"
    
    #here the shape of arr_ns is n rows by 400^2 pixels, so (n,160000)
    arr_ns = np.array([np.load(path_ns + "ns-map"+str(i)+".npy").flatten() for i in range(n)]) #no signal maps (y=0)
    
    arr_ws_noise = np.array([np.load(path_ws + "noise/noise"+str(i)+".npy").flatten() for i in range(n)])
    arr_ws_signal = np.array([np.load(path_ws + "signal/signal"+str(i)+".npy").flatten() for i in range(n)])
    
    #here we combine the signal map and noise maps. We can add these this way by construction of arr_ws_noise and arr_ns_signal
    arr_ws = amplitude*arr_ws_signal + arr_ws_noise #with signal maps (y=1)
    
    #labels 
    y = np.append(np.zeros(n), np.ones(n))
    
    #an array of the form (n + n, 400^2) where the first n maps contain no signal and the last n do.
    X = np.concatenate((arr_ns,arr_ws),axis=0)
    
    #we return a shuffled version of the data
    return shuffle(X, y, random_state=0)

In [38]:
X,y = get_data(100)

In [39]:
model = keras.Sequential([
    keras.layers.Dense(units=256, activation='relu', input_shape=(400*400,)),
    keras.layers.Dense(units=192, activation='relu'),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dense(units=32, activation='relu'),
    keras.layers.Dense(units=1, activation='sigmoid')
])

In [40]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [41]:
model.fit(X, y, validation_split=0.33, epochs=25, batch_size=20)

Epoch 1/25
7/7 [==============================] - 1s 160ms/step - loss: 0.8045 - accuracy: 0.5746 - val_loss: 1.1075 - val_accuracy: 0.5152
Epoch 2/25
7/7 [==============================] - 1s 142ms/step - loss: 0.6721 - accuracy: 0.9030 - val_loss: 1.1460 - val_accuracy: 0.5758
Epoch 3/25
7/7 [==============================] - 1s 141ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 1.4426 - val_accuracy: 0.5606
Epoch 4/25
7/7 [==============================] - 1s 146ms/step - loss: 0.0441 - accuracy: 0.9851 - val_loss: 1.5682 - val_accuracy: 0.5909
Epoch 5/25
7/7 [==============================] - 1s 142ms/step - loss: 0.0533 - accuracy: 0.9851 - val_loss: 1.7350 - val_accuracy: 0.5909
Epoch 6/25
7/7 [==============================] - 1s 137ms/step - loss: 1.1189e-05 - accuracy: 1.0000 - val_loss: 1.8767 - val_accuracy: 0.6667
Epoch 7/25
7/7 [==============================] - 1s 138ms/step - loss: 0.0416 - accuracy: 0.9925 - val_loss: 1.8631 - val_accuracy: 0.6515
Epoch 8/25
7/7 [